In [1]:
!pip search eli5 | grep INSTALLED || pip install eli5

  INSTALLED: 0.8.2 (latest)


In [2]:
!pip search regex | grep INSTALLED || pip install regex

  INSTALLED: 2019.6.8


In [3]:
from sklearn.datasets import fetch_20newsgroups

from sklearn.base import BaseEstimator, TransformerMixin
from keras.models import Model, Input
from keras.layers import Dense, LSTM, Dropout, Embedding, SpatialDropout1D, Bidirectional, concatenate
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score

from eli5.lime import TextExplainer

import regex as re
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [4]:
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.13.1


In [0]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers'),
)

twenty_test = fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers'),
)

In [0]:
class KerasTextClassifier(BaseEstimator, TransformerMixin):
#     '''Wrapper class for keras text classification models that takes raw text as input.'''
    def __init__(self, max_words=30000, input_length=100, emb_dim=20, n_classes=4, epochs=5, batch_size=32):
        self.max_words = max_words
        self.input_length = input_length
        self.emb_dim = emb_dim
        self.n_classes = n_classes
        self.epochs = epochs
        self.bs = batch_size
        self.model = self._get_model()
        self.tokenizer = Tokenizer(num_words=self.max_words+1, lower=True, split=' ', oov_token="UNK")
    def _get_model(self):
        input_text = Input((self.input_length,))
        text_embedding = Embedding(input_dim=self.max_words + 2, output_dim=self.emb_dim,
                                   input_length=self.input_length, mask_zero=False)(input_text)
        text_embedding = SpatialDropout1D(0.5)(text_embedding)
        bilstm = Bidirectional(LSTM(units=32, return_sequences=True, recurrent_dropout=0.5))(text_embedding)
        x = concatenate([GlobalAveragePooling1D()(bilstm), GlobalMaxPooling1D()(bilstm)])
        x = Dropout(0.7)(x)
        x = Dense(512, activation="relu")(x)
        x = Dropout(0.6)(x)
        x = Dense(512, activation="relu")(x)
        x = Dropout(0.5)(x)
        out = Dense(units=self.n_classes, activation="softmax")(x)
        model = Model(input_text, out)
        model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        return model
    def _get_sequences(self, texts):
        seqs = self.tokenizer.texts_to_sequences(texts)
        return pad_sequences(seqs, maxlen=self.input_length, value=0)
    def _preprocess(self, texts):
        return [re.sub(r"\d", "DIGIT", x) for x in texts]
    def fit(self, X, y):
# '''
#             Fit the vocabulary and the model.
#             :params:
#             X: list of texts.
#             y: labels.
# '''
        self.tokenizer.fit_on_texts(self._preprocess(X))
        self.tokenizer.word_index = {e: i for e,i in self.tokenizer.word_index.items() if i <= self.max_words}
        self.tokenizer.word_index[self.tokenizer.oov_token] = self.max_words + 1
        seqs = self._get_sequences(self._preprocess(X))
        self.model.fit(seqs, y, batch_size=self.bs, epochs=self.epochs, validation_split=0.1)
    def predict_proba(self, X, y=None):
        seqs = self._get_sequences(self._preprocess(X))
        return self.model.predict(seqs)
    def predict(self, X, y=None):
        return np.argmax(self.predict_proba(X), axis=1)
    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

In [0]:
text_model = KerasTextClassifier(epochs=20, max_words=20000, input_length=200)

In [8]:
text_model.fit(twenty_train.data, twenty_train.target);

Train on 2031 samples, validate on 226 samples
Epoch 1/20
2031/2031 [==============================] - 63s 31ms/step - loss: 1.3768 - acc: 0.2816 - val_loss: 1.3686 - val_acc: 0.3805
Epoch 2/20
2031/2031 [==============================] - 60s 30ms/step - loss: 1.3329 - acc: 0.3584 - val_loss: 1.2672 - val_acc: 0.4115
Epoch 3/20
2031/2031 [==============================] - 60s 30ms/step - loss: 1.1378 - acc: 0.4727 - val_loss: 0.8639 - val_acc: 0.6062
Epoch 4/20
2031/2031 [==============================] - 59s 29ms/step - loss: 0.7877 - acc: 0.6209 - val_loss: 0.6122 - val_acc: 0.6947
Epoch 5/20
2031/2031 [==============================] - 60s 30ms/step - loss: 0.6422 - acc: 0.6933 - val_loss: 0.5295 - val_acc: 0.7257
Epoch 6/20
2031/2031 [==============================] - 59s 29ms/step - loss: 0.5442 - acc: 0.7484 - val_loss: 0.5274 - val_acc: 0.7389
Epoch 7/20
2031/2031 [==============================] - 59s 29ms/step - loss: 0.4908 - acc: 0.7779 - val_loss: 0.4931 - val_acc: 0.7920
E

In [9]:
text_model.score(twenty_test.data, twenty_test.target)

0.7030625832223701

In [10]:
twenty_test.data[2]

'In <1993Apr29.112642.1@vms.ocom.okstate.edu> chorley@vms.ocom.okstate.edu writes:\n\n>In article <35004@castle.ed.ac.uk>, gtclark@festival.ed.ac.uk (G T Clark) writes:\n>> msnyder@nmt.edu (Rebecca Snyder) writes:\n>> \n>>>Does anyone know how posionous centipedes and millipedes are? If someone\n>>>was bitten, how soon would medical treatment be needed, and what would\n>>>be liable to happen to the person?\n>> \n>>>(Just for clarification - I have NOT been bitten by one of these,  but my\n>>>house seems to be infested, and I want to know \'just in case\'.)\n>> \n>>>Rebecca\n>> \n>> \n>> \tMillipedes, I understand, are vegetarian, and therefore almost\n>> certainly will not bite and are not poisonous. Centipedes are\n>> carnivorous, and although I don\'t have any absolute knowledge on this, I\n>> would tend to think that you\'re in no danger from anything but a\n>> concerted assault by several million of them.\n>> \n>> \t\t\tG.\n>Not sure of this but I think some millipedes cause a toxi

In [11]:
twenty_test.target_names[twenty_test.target[2]]

'sci.med'

In [12]:
doc = twenty_test.data[2]
te = TextExplainer(random_state=42)
te.fit(doc, text_model.predict_proba)
te.show_prediction(target_names=twenty_train.target_names)

In [13]:
te.clf_

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',
              power_t=0.5,
              random_state=<mtrand.RandomState object at 0x7f61939867e0>,
              shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
              warm_start=False)

In [14]:
te.vec_

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w+\\b',
                tokenizer=None, vocabulary=None)